# ChemML Wrapper Input File Generator

This is a graphical user interface (gui) for the ChemML Wrapper.

* first run gui in the notebook with three lines of code:
<pre>
```python
from cheml import rungui
run = rungui()
run.gui(script='new')
```

</pre>

* next press Finish button when you are done and download the input file

In [4]:
from cheml import rungui
run = rungui()
run.gui(script='new')